# A Guide to TF Layers: Building a Convolutional Neural Network

* [MNIST tutorials](https://www.tensorflow.org/tutorials/layers)
<img src="https://user-images.githubusercontent.com/11681225/46912292-54460680-cfac-11e8-89a3-d8d1a4ec13ae.png" width="20%">

* training dataset: 60000
* test dataset: 10000
* one example: gray scale image with $28 \times 28$ size
* [`cnn_mnist.py`](https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/examples/tutorials/layers/cnn_mnist.py) 참고

In [1]:
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/lab4all/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Convolutional neural networks

* Convolutional layers
  * $\textrm{ReLU}({\bf x} * {\bf w} + {\bf b})$
  * $*$: convolution operator
* Pooling layers
  * down sampling: `max-pooling`, `average-pooling`
* Dense (fully connected) layers
  * $\textrm{ReLU}({\bf w} {\bf x} + {\bf b})$

### Structure of LeNet 5

<img width="90%" alt="lenet5" src="https://user-images.githubusercontent.com/11681225/46912300-a0914680-cfac-11e8-92fb-f1817267b4e4.png">


* Convolutional Layer #1: Applies 32 5x5 filters (extracting 5x5-pixel subregions), with ReLU activation function
* Pooling Layer #1: Performs max pooling with a 2x2 filter and stride of 2 (which specifies that pooled regions do not overlap)
* Convolutional Layer #2: Applies 64 5x5 filters, with ReLU activation function
* Pooling Layer #2: Again, performs max pooling with a 2x2 filter and stride of 2
* Dense Layer #1: 1,024 neurons, with dropout regularization rate of 0.4 (probability of 0.4 that any given element will be dropped during training)
* Dense Layer #2 (Logits Layer): 10 neurons, one for each digit target class (0–9).

## CNN model with `tf.layers` APIs

* [`tf.layers`](https://www.tensorflow.org/api_docs/python/tf/layers) 링크
* [`tf.layers.conv2d()`](https://www.tensorflow.org/api_docs/python/tf/layers/conv2d)
* `tf.layers.max_pooling2d()`
* `tf.layers.dense()`

In [2]:
tf.set_random_seed(219)

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                  pool_size=[2, 2],
                                  strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                  pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat,
                          units=1024,
                          activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4,
    training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(
    labels=labels,
    logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

### input layer

* 4-rank Tensor: `[batch_size, image_height, image_width, channels]`

### conv2d layer

* [`tf.layers.conv2d()`](https://www.tensorflow.org/api_docs/python/tf/layers/conv2d) API
* 필수 arguments
```
tf.layers.conv2d(
        inputs,
        filters,
        kernel_size)
```
* `inputs`: 4-rank Tensor: `[batch_size, image_height, image_width, channels]`
* `filters`: output filter의 갯수
* `kernel_size`: `[height, width]`
* `padding`: `"valid"` or `"same"` (case-insensitive)
  * `valid`: [32 x 32] -> [28 x 28] (`kernel_size`: 5)
  * `same`: [32 x 32] -> [32 x 32] (`kernel_size`: 5)
* `activation`
  * 기본값이 None
  * `tf.nn.relu`를 습관적으로 해주는게 좋음 (맨 마지막 레이어를 제외하고)

### maxpooling2d layer

* [`tf.layers.maxpooling2d()`](https://www.tensorflow.org/api_docs/python/tf/layers/max_pooling2d) API
* 필수 arguments
```
tf.layers.maxpooling2d(
        inputs,
        pool_size,
        strides)
```
* `inputs`: 4-rank Tensor: `[batch_size, image_height, image_width, channels]`
* `pool_size`: `[height, width]`
* `strides`: `[height, width]`

### dense layer

* [`tf.layers.dense()`](https://www.tensorflow.org/api_docs/python/tf/layers/dense) API
* 필수 arguments
```
tf.layers.dense(
        inputs,
        units)
```
* `inputs`: 2-rank Tensor: `[batch_size, features]`
* `units`: output node 갯수
* `conv2d`나 `maxpooling2d` 뒤에 `dense`레이어를 쓰려면 `inputs` 텐서의 차원을 맞춰줘야 한다.

### dropout layer

* [`tf.layers.dropout()`](https://www.tensorflow.org/api_docs/python/tf/layers/dropout) API
* 필수 arguments
```
tf.layers.dropout(
        inputs
        rate=0.5,
        training=False)
```
* `inputs`: 2-rank Tensor: `[batch_size, features]`
* `rate`: dropout rate
* `training`: `training` mode인지 아닌지 구분해주는 `argument`

### logits layer

* `softmax`를 하기 전에 score 값(raw value)을 전달해 주는 `layer`
* `activation`을 하지 않는게 중요
* `units`갯수는 class 갯수와 동일

### generate predictions

* `predicted class` for each example
  * [`tf.argmax`](https://www.tensorflow.org/api_docs/python/tf/argmax) 사용
* `probabilities` for each possible target class for each example
  * [`tf.nn.softmax`](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) 사용

### calculate loss

* multiclass classification problems
  * `cross_entropy` loss: $- \sum y \log \hat{y}$
  * `tf.losses.softmax_cross_entropy` API 사용

### Configure the Training Op

* stochastic gradient descent
* [Optimizers API](https://www.tensorflow.org/api_guides/python/train#Optimizers)
```
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(
      loss=loss,
      global_step=tf.train.get_global_step())
```

### Add evaluation metrics

```
eval_metric_ops = {
    "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
```

## Training

In [3]:
# Load training and eval data from tf.keras
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data / 255.
train_data = train_data.reshape(-1, 784)
train_labels = np.asarray(train_labels, dtype=np.int32)

test_data = test_data / 255.
test_data = test_data.reshape(-1, 784)
test_labels = np.asarray(test_labels, dtype=np.int32)

11493376/11490434 [==============================] - 1s 0us/step


In [4]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="graphs/01.1.mnist.deep.with.estimator")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'graphs/01.1.mnist.deep.with.estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f70a9bf2630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [6]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=32,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=100,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into graphs/01.1.mnist.deep.with.estimator/model.ckpt.
INFO:tensorflow:probabilities = [[0.09946432 0.10026968 0.1001089  0.1088763  0.09241221 0.10655838
  0.09769787 0.09167217 0.09558441 0.10735577]
 [0.10443639 0.09028919 0.10932202 0.10557002 0.08750195 0.1129559
  0.09283712 0.10187628 0.0911648  0.10404633]
 [0.08729037 0.08273202 0.11882669 0.10135613 0.09783328 0.10637151
  0.08952356 0.11445157 0.09460464 0.10701024]
 [0.09924364 0.0940272  0.11895797 0.11179941 0.095858   0.09456187
  0.0971493  0.09239972 0.0989261  0.09707679]
 [0.09159164 0.09455547 0.10412133 0.10369674 0.1053172  0.11078608
  0.08988277 0.09075892 0.09818998 0.11109987]
 [0.10220621 0.10478947 0.11513246 0.09837164 0.09952052 0.09

In [7]:
# Evaluate the model and print results
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

test_results = mnist_classifier.evaluate(input_fn=test_input_fn)
print(test_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-04-08:48:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from graphs/01.1.mnist.deep.with.estimator/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-04-08:48:03
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.9118, global_step = 100, loss = 0.29634535
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: graphs/01.1.mnist.deep.with.estimator/model.ckpt-100
{'accuracy': 0.9118, 'loss': 0.29634535, 'global_step': 100}
